In [1]:
import os
import pickle
import numpy as np
from queries import Simbad
from astropy.io import fits
from astropy.coordinates import SkyCoord
from IPython.display import clear_output

top_path='/DataDisk/datafiles/PGCC_HST/'

def find_in_tgas(key,field):
    filenames=[top_path+'gaia_data/TgasSource_000-000-0'+'{:02}'.format(i)+'.fits' for i in range(16)]
    for filename in filenames:
        gaia_data=fits.open(filename)[1].data
        #gaia_entry=gaia_data[gaia_data[field]==key]
        gaia_entry=[i for i,item in enumerate(gaia_data[field]==key) if item==True]
        if len(gaia_entry)>0:
            return filenames.index(filename),gaia_entry[0]
    return None
        
#Returns angular separation between two Ra/Dec coordinates in degrees
def get_ang_sep(ra1,dec1,ra2,dec2):
    ra1_rad=ra1*np.pi/180.
    dec1_rad=dec1*np.pi/180.
    ra2_rad=ra2*np.pi/180.
    dec2_rad=dec2*np.pi/180.

    del_ra=ra2_rad-ra1_rad
    del_ras=np.sin(del_ra)
    del_rac=np.cos(del_ra)
    dec1s=np.sin(dec1_rad)
    dec2s=np.sin(dec2_rad)
    dec1c=np.cos(dec1_rad)
    dec2c=np.cos(dec2_rad)
    result=(180./np.pi)*np.arctan(np.sqrt((dec2c**2)*(del_ras**2)+(dec1c*dec2s-dec1s*dec2c*del_rac)**2)/(dec1s*dec2s+dec1c*dec2c*del_rac))
    if result < 0:
        return result+180.
    else:
        return result
    
    
    
def store_results(results):
    filename='sightline_pgcc_gaia_results.txt'
    with open(filename,'w') as myfile:
        for entry in results:
            myfile.write('|'.join([entry[0],
                                   str(entry[1][0].value)+' '+str(entry[1][1].value),
                                   str(entry[2]),
                                   str(entry[3]),
                                   '\n']))
        

In [17]:
x1d_path=top_path+'x1d_files/'
sightlines=[x for x in os.listdir(x1d_path) 
            if os.path.isdir(x1d_path+x+'/E140H')==True and x.startswith('.')==False]

pgcc_data=fits.open(top_path+'HFI_PCCS_GCC_R2.02.fits')[1].data

In [18]:
sightline_pgcc_gaia_data=[]

for sightline in sightlines:
    clear_output()
    print sightlines.index(sightline)+1, '/', len(sightlines)
    coords=Simbad.Position(sightline)
    nearest=9999
    nearest_pgcc=[]
    for i in range(len(pgcc_data)):
        pgcc_ra,pgcc_dec=pgcc_data[i][3:5]
        if abs(coords[0].value-pgcc_ra)<10 and abs(coords[1].value-pgcc_dec)<10:
            angsep=get_ang_sep(coords[0].value,coords[1].value,pgcc_ra,pgcc_dec)
            if angsep<nearest:
                nearest=angsep
                nearest_pgcc=i
    
    
    idlist=Simbad.IDList(sightline)
    hip=[x for x in idlist if x.startswith('HIP')]
    tyc=[x for x in idlist if x.startswith('TYC')]
    if len(hip)>0 or len(tyc)>0:
        if len(hip)>0:
            hip_id=int(hip[0].split()[1])
            gaia_entry=find_in_tgas(hip_id,'hip')
        elif len(tyc)>0:
            tyc_id=tyc[0].split()[1]
            gaia_entry=find_in_tgas(tyc_id,'tycho2_id')
    
    sightline_pgcc_gaia_data.append([sightline,coords,nearest_pgcc,gaia_entry])


94 / 94


In [19]:
store_results(sightline_pgcc_gaia_data)